In [134]:
from dataclasses import dataclass
import pandas as pd
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

In [ ]:
data.head()


In [187]:
target = (data.y == 'yes')
raw_features = data.drop(['y', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed','pdays'],axis=1)
features = pd.get_dummies(raw_features)

In [ ]:
features.head()

In [188]:
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=.20,train_size=.80,random_state=1000)

In [189]:
clf = DecisionTreeClassifier(max_depth=7,criterion="entropy")

clf.fit(X_train,y_train)

clf.score(X_test,y_test)

y_predicted = clf.predict(X_test)
#test accuracy
metrics.accuracy_score(y_test, y_predicted)

0.8965768390386016

In [190]:
from pandas.core.common import random_state
classifier_RF = RandomForestClassifier(n_estimators=120,max_depth=5,oob_score=True,random_state=20000)

classifier_RF.fit(X_train,y_train)

y_predicted_RF = classifier_RF.predict(X_test)
#test accuracy
metrics.accuracy_score(y_test, y_predicted_RF)


0.8977907259043457

In [ ]:
print(metrics.classification_report(y_test,y_predicted))

In [ ]:
from sklearn.metrics._plot.confusion_matrix import plot_confusion_matrix
confusion = plot_confusion_matrix(clf,X_test,y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(85, 20))
tree.plot_tree(clf, fontsize=15, feature_names=features.columns)
plt.show()

Linear Discriminent


In [192]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [195]:
clf_linear = LinearDiscriminantAnalysis()
clf_linear.fit(X_train,y_train)
y_predicted_LN = clf_linear.predict(X_test)
print(metrics.classification_report(y_test,y_predicted_LN))

              precision    recall  f1-score   support

       False       0.91      0.97      0.94      7320
        True       0.53      0.27      0.36       918

    accuracy                           0.89      8238
   macro avg       0.72      0.62      0.65      8238
weighted avg       0.87      0.89      0.88      8238

